<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv3/blob/main/4_adverserial_attacks_drebin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/adverserial_attacks_functions.py',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 3.53MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 1.31MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 1.40MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 1.22MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 7.08MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *

torch.manual_seed(0)



cpu
cpu


In [4]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [5]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [6]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

Epoch: 1/50 | Training loss (epoch level): 0.4479 | Train accuracy: 92.48%
	Val accuracy(without attack) 96.63% and accuracy(with attack) 72.02% under attack and overall accuracy 84.33%.
	Model select at epoch 1 with validation accuracy 84.33% and accuracy 72.02% under attack.
Epoch: 2/50 | Training loss (epoch level): 0.1073 | Train accuracy: 96.83%
	Val accuracy(without attack) 98.17% and accuracy(with attack) 86.98% under attack and overall accuracy 92.58%.
	Model select at epoch 2 with validation accuracy 92.58% and accuracy 86.98% under attack.
Epoch: 3/50 | Training loss (epoch level): 0.0607 | Train accuracy: 98.08%
	Val accuracy(without attack) 98.28% and accuracy(with attack) 91.32% under attack and overall accuracy 94.8%.
	Model select at epoch 3 with validation accuracy 94.8% and accuracy 91.32% under attack.
Epoch: 4/50 | Training loss (epoch level): 0.0498 | Train accuracy: 98.47%
	Val accuracy(without attack) 98.69% and accuracy(with attack) 86.15% under attack and overal

In [ ]:
# with weighted loss
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT, train_loader, val_loader, adv_epochs=50,lr=0.005, attack=dfgsm_k, **attack_param)

Epoch: 1/50 | Training loss (epoch level): 0.3193 | Train accuracy: 85.24%
	Val accuracy(without attack) 96.34% and accuracy(with attack) 92.52% under attack and overall accuracy 94.43%.
	Model select at epoch 1 with validation accuracy 94.43% and accuracy 92.52% under attack.
Epoch: 2/50 | Training loss (epoch level): 0.1226 | Train accuracy: 96.32%
	Val accuracy(without attack) 96.81% and accuracy(with attack) 97.05% under attack and overall accuracy 96.93%.
	Model select at epoch 2 with validation accuracy 96.93% and accuracy 97.05% under attack.
Epoch: 3/50 | Training loss (epoch level): 0.0919 | Train accuracy: 97.24%
	Val accuracy(without attack) 97.28% and accuracy(with attack) 96.12% under attack and overall accuracy 96.7%.
	Model select at epoch 2 with validation accuracy 96.93% and accuracy 97.05% under attack.
Epoch: 4/50 | Training loss (epoch level): 0.1095 | Train accuracy: 96.62%
	Val accuracy(without attack) 97.95% and accuracy(with attack) 91.69% under attack and overa

In [7]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [8]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
model_evaluation(model_DNN, test_loader, device)

Test Accuracy: 0.9912
Test balanced Accuracy: 0.9785
Precision: 0.9637
Recall: 0.9619
F1-score: 0.9628
True Positives (TP): 1087
True Negatives (TN): 8391
False Positives (FP): 41
False Negatives (FN): 43
False Negative Rate (FNR): 3.8053
False Positive Rate (FPR): 0.4862


In [10]:
model_evaluation(model_AT_rFGSM,test_loader,device)

Test Accuracy: 0.9893
Test balanced Accuracy: 0.9710
Precision: 0.9622
Recall: 0.9469
F1-score: 0.9545
True Positives (TP): 1070
True Negatives (TN): 8390
False Positives (FP): 42
False Negatives (FN): 60
False Negative Rate (FNR): 5.3097
False Positive Rate (FPR): 0.4981


In [11]:
model_evaluation(model_AT_rFGSM_weightedLoss,test_loader,device)

Test Accuracy: 0.9872
Test balanced Accuracy: 0.9782
Precision: 0.9286
Recall: 0.9664
F1-score: 0.9471
True Positives (TP): 1092
True Negatives (TN): 8348
False Positives (FP): 84
False Negatives (FN): 38
False Negative Rate (FNR): 3.3628
False Positive Rate (FPR): 0.9962


In [12]:
# rFGSM
attack_params = {"k":50, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)


Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.81%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.84%.


In [ ]:
# rFGSM
attack_params = {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.98%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.22%.


In [ ]:
adv_predict(test_loader, model_AT_rFGSM, attack=pgd, is_report_loss_diff=False, device=device)

Adversarial accuracy (without attack): 99.36% | Under attack: 90.83%.


In [ ]:
# Groose
attack_params = {"k":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, grosse_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, grosse_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, grosse_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 90.09%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 80.27%.


In [ ]:
# BCA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bca_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bca_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, bca_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 90.09%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 77.43%.


In [ ]:
# BGA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bga_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bga_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, bga_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.81%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 94.16%.


In [ ]:
# rFGSM
attack_params = {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, dfgsm_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.89%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.22%.


In [ ]:
# PGD-l1
attack_params = {"k":500, "step_length":1., 'norm':'l1', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 90.09%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 77.43%.


In [ ]:
# PGD-l2
attack_params = {"k":200, "step_length":0.05, 'norm':'l2', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 31.24%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 94.69%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 96.64%.


In [ ]:
# PGD-linf
attack_params = {"k":500, "step_length":0.002, 'norm':'linf', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 90.35%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 90.53%.


In [ ]:
# Mimicry×1
attack_params = {"trials":1, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 95.04%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 94.6%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 96.37%.


In [ ]:
# Mimicry×10
attack_params = {"trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 94.16%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.04%.


In [ ]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.63%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 94.34%.


In [ ]:
# Mimicry×100
attack_params = {"trials":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 24.6%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.39%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 92.3%.


In [ ]:
# Mimicry×1000
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 2.478%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.04%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 87.26%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.001}, "steps":500}
adv_predict(test_loader, model_AT_rFGSM, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 0.000%.
step-wise max: attack effectiveness 10.000%.


KeyboardInterrupt: 

In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),

    (pgd, {'k': 100, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'k': 100, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (StepwiseMax_onestep2, {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

    # Add more attacks as needed
]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    print()  # Print an empty line for separation


Running attack: dfgsm_k with parameters: {'k': 50, 'epsilon': 0.02, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 83.43%.

Running attack: bga_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.57%.

Running attack: bca_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: grosse_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: pgd with parameters: {'k': 100, 'step_length': 1.0, 'norm': 'l1', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.24%.

Running attack: pgd with parameters: {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.67%.

Running att